In [1]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import torch
from notebooks.experiment_robust import load_robust_experiment
from notebooks.experiment_torch import load_torchvision_experiment

model, dl_train, dl_eval = load_robust_experiment("Salman2020Do_R50", "imagenet")

/home/fre.gilad/source/ulib/.venv/lib/python3.12/site-packages/robustbench/loaders.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


ImportError: cannot import name 'eval' from partially initialized module 'ulib' (most likely due to a circular import) (/home/fre.gilad/source/ulib/ulib/__init__.py)

In [ ]:
from ulib import StopCriteria, PertModule
from ulib.attacks.usgd import USGD

stop = StopCriteria(max_epochs=10, max_time=60 * 10, patience=5, patience_delta=0.001)
pert_model = PertModule(model, data_shape=dl_train.get_tensor(0).shape[1:], eps=16 / 255, random_init=True)
optimizer = torch.optim.Adam(pert_model.parameters(), lr=1e-2, maximize=True)
criterion = torch.nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=1)
grad_scaler = torch.GradScaler(device=pert_model.device.type)

attack = USGD(
    pert_model=pert_model,
    optimizer=optimizer,
    criterion=criterion,
    skip_already_fooled=False,
    scheduler=scheduler,
    grad_scaler=grad_scaler,
    logging_enable=True,
)

pert = attack.fit(dl_train, dl_eval, stop)

In [ ]:
attack.close()

In [ ]:
from ulib.evaluator import ExtendedEvaluator
from ulib.utils.visualize import display_pert

evaluator = ExtendedEvaluator(pert_model, verbose=True)
print(evaluator.evaluate(dl_eval))
display_pert(pert_model)